In [3]:
import requests, json
import numpy
from numpy import arctanh
from skimage import io
from io import BytesIO
import math
from math import *
import matplotlib.pyplot as plt
%matplotlib inline
import collections as cl
import osmium

class isInterVisible():
    
    def __init__(self,z):
        self.z = z
    
    def export_json(self,latlon_list,output_file):
        ys = cl.OrderedDict()
        ys["type"] = "FeatureCollection"
        ys["features"] = []
        
        if'true' in output_file:
            LOS = "LOS:True"
        else:
            LOS = "LOS:False"
        
        for i in range(len(latlon_list)):
            data = cl.OrderedDict()
            data["properties"] = {"Name":LOS}
            data["type"] = "Feature"
            data["geometry"] = {"type":"Point","coordinates":latlon_list[i]}
            ys["features"].append(data)

        fw = open(output_file,'w')
        json.dump(ys,fw,indent=4)
        
    def vincenty_inverse(self, lat1, lon1, lat2, lon2, ellipsoid=None):
        """
        緯度経度で指定した2地点間の距離を返す
        """
        # 楕円体
        ELLIPSOID_GRS80 = 1 # GRS80
        ELLIPSOID_WGS84 = 2 # WGS84

        # 楕円体ごとの長軸半径と扁平率
        GEODETIC_DATUM = {
            ELLIPSOID_GRS80: [
                6378137.0,         # [GRS80]長軸半径
                1 / 298.257222101, # [GRS80]扁平率
            ],
            ELLIPSOID_WGS84: [
                6378137.0,         # [WGS84]長軸半径
                1 / 298.257223563, # [WGS84]扁平率
            ],
        }

        # 反復計算の上限回数
        ITERATION_LIMIT = 1000

        # 差異が無ければ0.0を返す
        if isclose(lat1, lat2) and isclose(lon1, lon2):
            return 0.0

        # 計算時に必要な長軸半径(a)と扁平率(ƒ)を定数から取得し、短軸半径(b)を算出する
        # 楕円体が未指定の場合はGRS80の値を用いる
        a, ƒ = GEODETIC_DATUM.get(ellipsoid, GEODETIC_DATUM.get(ELLIPSOID_GRS80))
        b = (1 - ƒ) * a

        φ1 = radians(lat1)
        φ2 = radians(lat2)
        λ1 = radians(lon1)
        λ2 = radians(lon2)

        # 更成緯度(補助球上の緯度)
        U1 = atan((1 - ƒ) * tan(φ1))
        U2 = atan((1 - ƒ) * tan(φ2))

        sinU1 = sin(U1)
        sinU2 = sin(U2)
        cosU1 = cos(U1)
        cosU2 = cos(U2)

        # 2点間の経度差
        L = λ2 - λ1

        # λをLで初期化
        λ = L

        # 以下の計算をλが収束するまで反復する
        # 地点によっては収束しないことがあり得るため、反復回数に上限を設ける
        for i in range(ITERATION_LIMIT):
            sinλ = sin(λ)
            cosλ = cos(λ)
            sinσ = sqrt((cosU2 * sinλ) ** 2 + (cosU1 * sinU2 - sinU1 * cosU2 * cosλ) ** 2)
            cosσ = sinU1 * sinU2 + cosU1 * cosU2 * cosλ
            σ = atan2(sinσ, cosσ)
            sinα = cosU1 * cosU2 * sinλ / sinσ
            cos2α = 1 - sinα ** 2
            cos2σm = cosσ - 2 * sinU1 * sinU2 / cos2α
            C = ƒ / 16 * cos2α * (4 + ƒ * (4 - 3 * cos2α))
            λʹ = λ
            λ = L + (1 - C) * ƒ * sinα * (σ + C * sinσ * (cos2σm + C * cosσ * (-1 + 2 * cos2σm ** 2)))

            # 偏差が.000000000001以下ならbreak
            if abs(λ - λʹ) <= 1e-12:
                break
        else:
            # 計算が収束しなかった場合はNoneを返す
            return None

        # λが所望の精度まで収束したら以下の計算を行う
        u2 = cos2α * (a ** 2 - b ** 2) / (b ** 2)
        A = 1 + u2 / 16384 * (4096 + u2 * (-768 + u2 * (320 - 175 * u2)))
        B = u2 / 1024 * (256 + u2 * (-128 + u2 * (74 - 47 * u2)))
        Δσ = B * sinσ * (cos2σm + B / 4 * (cosσ * (-1 + 2 * cos2σm ** 2) - B / 6 * cos2σm * (-3 + 4 * sinσ ** 2) * (-3 + 4 * cos2σm ** 2)))

        # 2点間の楕円体上の距離
        s = b * A * (σ - Δσ)

        # 各点における方位角
        α1 = atan2(cosU2 * sinλ, cosU1 * sinU2 - sinU1 * cosU2 * cosλ)
        α2 = atan2(cosU1 * sinλ, -sinU1 * cosU2 + cosU1 * sinU2 * cosλ) + pi

        if α1 < 0:
            α1 = α1 + pi * 2

        return s / 1000  #距離(km)

    def get_combined_image(self, topleft_x, topleft_y, size=1):
        """
        指定した範囲のタイルを結合して取得
        """
        rows = []
        blank = numpy.zeros((256, 256, 4), dtype=numpy.uint8)

        for y in range(size):
            row = []
            for x in range(size):
                try:
                    img = self.get_astergdem2_dsm(self.z, topleft_x + x, topleft_y + y)
                except Exception as e:
                    img = blank

                row.append(img)
            rows.append(numpy.hstack(row))
        return  numpy.vstack(rows)


    # 標高タイル（ASTER GDEM 2.0）を取得
    def get_astergdem2_dsm(self, zoom, xtile, ytile):
        """
        ASTER GDEM2 の標高タイルを取得
        """
        url = " https://gisapi.tellusxdp.com/astergdem2/dsm/{}/{}/{}.png".format(zoom, xtile, ytile)
        headers = {
            "Authorization": "Bearer " + TOKEN
        }

        r = requests.get(url, headers=headers)
        return io.imread(BytesIO(r.content))

    def latlon2tile(self, lon, lat, z):
        """
        指定した緯度経度からピクセル座標を取得
        https://qiita.com/kobakou/items/4a5840542e74860a6b1b
        """
        L = 85.05112878
        x = int((lon/180 + 1) * 2**(z+7))
        y = int( (2**(z+7) / pi * ( -arctanh(sin(pi*lat/180)) + arctanh(sin(pi*L/180)) ) ))

        return [y, x]

    def num2deg(self, xtile, ytile, zoom):
        """
        この関数は引数に ( x、y、z ) を指定することで、タイルの左上の角の緯度経度を返します。
        他にも、xとy の両方もしくは片方に、+1 した引数を指定すると別の角の座標を取得でき、
        +0.5 で中央の座標を取得できます。
        https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Python
        """
        n = 2.0 ** zoom
        lon_deg = xtile / n * 360.0 - 180.0
        lat_rad = atan(sinh(pi * (1 - 2 * ytile / n)))
        lat_deg = degrees(lat_rad)
        return (lon_deg, lat_deg)

    def get_radian_btw_p1p2(self, y1, x1, y2, x2):
        """
        2点間のかたむき(y=ax)を取得する
        """
        #0で割ると division by zero エラーが出る
        if (x2 - x1) == 0:
            return "vertical"
        elif (y2 - y1) == 0:
            return "horizontal"
        else:
            return (y2 - y1) / (x2 - x1)

    def get_heights_btw_p1p2(self, tile, y1, x1, y2, x2, radian):
        """
        始点から終点を結ぶ直線上の、標高データを取得
        一次関数：y = ax + b

        Parameters
        ----------
        tile : ndarray
            標高タイル
        y1 : number 
            点Aのy座標
        x1 : number 
            点AのX座標
        radian : number 
            傾き
        Returns
        -------
        actual_heights : ndarray
            x1y1を通り、かたむきradian上の標高データ
        """
        if radian == "vertical":
            return tile[:,y1]
        elif radian == "horizontal":
            return tile[x1,:]

        #原点を設定
        #x座標が小さい点をoriginに設定
        if x1 <= x2:
            reverseFlag = True
            origin_y = y1
            origin_x = x1
            point_y = y2
            point_x = x2
        else:
            reverseFlag = False
            origin_y = y2
            origin_x = x2
            point_y = y1
            point_x = x1

        max_index = len(tile) -1
        radian_tile = []

        while (round(origin_y) <= max_index and round(origin_x) <= max_index) and (round(origin_y) != point_y or round(origin_x) != point_x):
            #round関数で平均化
            radian_tile.append(tile[round(origin_y),round(origin_x)])
            origin_y += radian
            origin_x += 1

        if reverseFlag:
            radian_tile.reverse()
        
        return numpy.array(radian_tile)
        
        
    def calc_height_chiriin_style(self, R, G, B, u=1):
        """
        標高タイルのRGB値から標高を計算する
        """
        hyoko = int(R*256*256 + G * 256 + B)
        if hyoko == 8388608:
            raise ValueError('N/A')
        if hyoko > 8388608:
            hyoko = (hyoko - 16777216)/u
        if hyoko < 8388608:
            hyoko = hyoko/u
        return hyoko

    def plot_height(self, actual_heights, u=1):
        """
        高度のグラフをプロットする
        """
        heights = []
        heights_err = []

        for k in range(len(actual_heights)):
            R = actual_heights[k,0]
            G = actual_heights[k,1]
            B = actual_heights[k,2]
            try:
                heights.append(self.calc_height_chiriin_style(R, G, B, u))
            except ValueError as e:
                heights.append(0)
                heights_err.append((i,j))

        fig, ax= plt.subplots(figsize=(8, 4)) 
        plt.plot(heights)
        ax.text(0.01, 0.95, 'Errors: {}'.format(len(heights_err)), transform=ax.transAxes)
        plt.show()

    def worldpx2tilepx(self, worldpx,left_top_bbx_px):
        """
        世界座標をタイル内座標に変換する
        """
        y1 = worldpx[0] - left_top_bbx_px[0] -1
        x1 = worldpx[1] - left_top_bbx_px[1] -1
        return [y1,x1]

    def calc_max_theta(self, distance,earth_r):
        """
        観測点と対象点の角度（radian）を返す
        """
        return distance/(2 *earth_r)

    def getHorizonDistance_km(self, h0_km,earth_r):
        return sqrt(pow(h0_km, 2) + 2 * earth_r * h0_km);

    def getTargetHiddenHeight_km(self, d2_km, earth_r):
        if d2_km < 0:
            return 0;
        return sqrt(pow(d2_km, 2) + pow(earth_r, 2)) - earth_r;

    def calculate_shizumikomi(self, h0,d0,earth_r):
        """
        h0 : 観測点の標高
        d0 : 対象点との距離 km
        """
        h0_km = h0 * 0.001
        d0_km = d0

        d1_km = self.getHorizonDistance_km(h0_km,earth_r);
        h1_m = self.getTargetHiddenHeight_km(d0_km - d1_km,earth_r) * 1000;

        return h1_m

    def km_per_tile(self, d0_km,tileNum):
        """
        1タイルの辺の長さ(km)を取得
        """
        return d0_km/tileNum

    def get_tile_num(self, lat, lon, zoom):
        """
        緯度経度からタイル座標を取得する
        """
        # https://wiki.openstreetmap.org/wiki/Slippy_map_tilenames#Python
        lat_rad = math.radians(lat)
        n = 2.0 ** zoom
        xtile = int((lon + 180.0) / 360.0 * n)
        ytile = int((1.0 - math.log(math.tan(lat_rad) + (1 / math.cos(lat_rad))) / math.pi) / 2.0 * n)
        return (xtile, ytile)

    def get_smaller_tile_axis(self, observer_tile,target_tile,axis):

        if axis == "x":
            index = 0
        elif axis == "y":
            index = 1

        if observer_tile[index] > target_tile[index]:
            smaller = target_tile[index]
        elif target_tile[index] > observer_tile[index]:
            smaller = observer_tile[index]
        else:
            smaller = observer_tile[index]

        return smaller

    def get_origin_tile(self, observer_tile,target_tile):

        x_smaller = self.get_smaller_tile_axis(observer_tile,target_tile,"x")
        y_smaller = self.get_smaller_tile_axis(observer_tile,target_tile,"y")

        return [x_smaller,y_smaller]

    def get_bigger_diff(self, observer_tile,target_tile):

        x_diff = abs(observer_tile[0] - target_tile[0])
        y_diff = abs(observer_tile[1] - target_tile[1])

        if x_diff > y_diff:
            return x_diff
        elif y_diff > x_diff:
            return y_diff
        else:
            return x_diff
        
    def calc_intervisibility(self, target, observer, losTrueArray, losFalseArray ):
        # 緯度と経度に分割
        lon1=target[1]
        lat1=target[0]
        lon2=observer[1]
        lat2=observer[0]

        #タイル座標取得
        target_tile = self.get_tile_num(lat1,lon1,self.z)
        observer_tile = self.get_tile_num(lat2,lon2,self.z)

        #始点を取得：観測点と対象点を角とする四角形の左上の座標
        origin_tile = self.get_origin_tile(observer_tile,target_tile)

        #上の四角形で長辺のタイル数を取得
        diff = self.get_bigger_diff(observer_tile,target_tile)

        ##タイルを取得
        combined_img = self.get_combined_image(origin_tile[0], origin_tile[1], diff+1)

        #地球の曲率を含めた2点間の距離
        distance_km = self.vincenty_inverse(lat1, lon1, lat2, lon2, 1)

        ##世界ピクセル座標を取得
        left_top_bbx = self.num2deg(origin_tile[0], origin_tile[1], self.z)
        left_top_bbx_px = self.latlon2tile(left_top_bbx[0], left_top_bbx[1], self.z) #始点

        target_wpx = self.latlon2tile(lon1, lat1, self.z) #対象点
        observer_wpx = self.latlon2tile(lon2, lat2, self.z) #観測点

        ##タイル内ピクセル座標を取得
        target_tpx = self.worldpx2tilepx(target_wpx,left_top_bbx_px)#対象点
        target_y = target_tpx[0]
        target_x = target_tpx[1]
        observer_tpx = self.worldpx2tilepx(observer_wpx,left_top_bbx_px) #観測点
        observer_y = observer_tpx[0]
        observer_x = observer_tpx[1]

        ##2点間のかたむきを取得
        radian = self.get_radian_btw_p1p2(target_y, target_x, observer_y, observer_x)
        
        ##2点間の標高データを取得（単位:m）
        actual_heights = self.get_heights_btw_p1p2(combined_img, target_y, target_x, observer_y, observer_x, radian) 

        #2点のカラーコードを取得
        target_color = combined_img[target_y][target_x]
        observer_color = combined_img[observer_y][observer_x]

        #2点の標高を取得（単位:m）
        u=1
        target_height = round(self.calc_height_chiriin_style(target_color[0],target_color[1],target_color[2],u),2) # 対象点
        observer_height = round(self.calc_height_chiriin_style(observer_color[0],observer_color[1],observer_color[2],u),2) #観測点

        #沈み込み量を取得（単位:m）
        shizumikomi = self.calculate_shizumikomi(observer_height,distance_km,earth_r)

        #対象点の見た目の標高を取得（単位:m）
        h1_mitame = target_height - shizumikomi

        #タイル当たりの辺の長さを取得（km）
        tile_km = self.km_per_tile(distance_km,len(actual_heights))

        #標高で取得したタイル数分ループを回す
        for indexNum in range(len(actual_heights)):

            #中間の標高を取得
            hyoko = round(self.calc_height_chiriin_style( actual_heights[indexNum][0], actual_heights[indexNum][1], actual_heights[indexNum][2],u),2)
            
            #沈み込み量を取得
            d0 = index*tile_km
            shizumikomi = self.calculate_shizumikomi(observer_height,d0,earth_r)
            
            #中間の見た目の標高を取得
            h2_mitame = hyoko - shizumikomi

            #見た目の高さで比較
            distance_km = round(distance_km,3)
            if h1_mitame < 0:
                message="見通し：不可（地平線）"
                result=False
                break;
            elif h1_mitame < h2_mitame: 
                message='''
            見通し：不可（障害物有り）
            障害物までの距離：{}(km)'''.format(d0)
                result=False
                break;
            else:
                message="見通し：可"
                result=True
                
        print(result)
        # 配列に代入
        if result:
            los_true.append([lon1,lat1])
        else:
            los_false.append([lon1,lat1])

class getBuildings(osmium.SimpleHandler):
    
    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.num_nodes = 0
        self.num_ways = 0
        self.num_building = 0
        self.nodes = []
        self.nodeId_of_buildings = []

    def count_building(self, w):
        if 'building' in w.tags:
            # way（複数のノードで構成されている要素）の1つのノードIDを抽出
            self.nodeId_of_buildings.append(w.nodes[0].ref)
            self.num_building += 1
        
    def node(self, n):
        self.num_nodes += 1
        self.nodes.append([n.id,n.location])

    def way(self, w):
        self.num_ways += 1
        self.count_building(w)


#ご自身のAPIキーを設定してください
#参考：https://sorabatake.jp/5048
TOKEN = 'YOUR-TOKEN'

#地球の半径(km)
earth_r = 8494.666; #光の屈折を踏まえた地球の半径4/3倍（ここでは光の屈折率を電磁波と同じと見なす） （実際の半径は6371km）

#見通しを判定したい任意の地点を緯度経度で指定
observer = [34.751888,135.237224] #六甲山展望台

##
# 緯度経度を含む、Buldingノードの配列を生成
## 
GB = getBuildings()
#処理するデータを追加
filename = "example"
GB.apply_file(filename + ".pbf")
print("Number of nodes: %d" % GB.num_nodes)
print("Number of ways: %d" % GB.num_ways)
print("Number of building: %d" % GB.num_building)
building = []

for node in GB.nodes:
    if node[0] in GB.nodeId_of_buildings:
        building.append(node)

#LOSをチェック
IV = isInterVisible(12)
index=0;
los_true_bulding=0;
los_true = []
los_false = []
base=10000000

for node in building:
    target = [int(node[1].y)/base, int(node[1].x)/base]
    result = IV.calc_intervisibility(target,observer,los_true,los_false)
    index+=1
    if(result):
        los_true_bulding += 1
IV.export_json(los_true,filename + "_los_true.geojson")
IV.export_json(los_false,filename + "_los_false.geojson")

print("Number of los true: %d" % len(los_true))
print("Number of los false: %d" % len(los_false))
print("Geojson files are exported.")

Number of nodes: 74843
Number of ways: 9607
Number of building: 6057
False
True
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fal